<a href="https://colab.research.google.com/github/AaronJing/FreezeMLP/blob/main/mlpfreezing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-e7440cb7-6b4b-a4ad-dc2c-03acbc84df7a)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
batch_size = 64

In [ ]:
# Prepare data set
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(
    y_train, num_classes=10, dtype='uint8')
y_test = tf.keras.utils.to_categorical(
    y_test, num_classes=10, dtype='uint8')


In [ ]:
# augmentation
from sklearn.model_selection import train_test_split 
x_train, x_valid, y_train, y_valid = train_test_split(x_train ,y_train, test_size=0.1, 
                                                            stratify=y_train, 
                                                            random_state=40, shuffle = True)
train_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(samplewise_std_normalization=True ,zoom_range=0.2, 
                                                                width_shift_range=0.1, 
                                                                height_shift_range = 0.1, 
                                                                horizontal_flip=True)
valid_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(samplewise_std_normalization=True)
test_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(samplewise_std_normalization=True)
train_set_conv = train_DataGen.flow(x_train, y_train, batch_size=batch_size) # train_lab is categorical 
valid_set_conv = valid_DataGen.flow(x_valid, y_valid, batch_size=batch_size) # so as valid_lab
test_set_conv = test_DataGen.flow(x_test, y_test, batch_size=batch_size) # so as valid_lab

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:

class model_16():
  def dense(name, neurons, kernel_initializer='glorot_uniform',activation='tanh'):
    return layers.Dense(neurons, activation = activation, name = name, kernel_initializer = kernel_initializer)
  inputs = keras.Input(shape=(32,32,3),name = "InputLayer")
  x = layers.Flatten()(inputs)
  i = 0 
  while i < 15:
    if i in [3,7,11]:
      x = dense('dense'+str(i),500)(x)
    else:
      x = dense('dense'+str(i),500, tf.keras.initializers.Identity())(x)
    i += 1

  x = layers.Dense(10, name='dense_logits')(x)
  outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
  def __init__(self):
    self.model = keras.Model(inputs=self.inputs, outputs=self.outputs)
    self.compile()
  def compile(self):
    self.model.compile(loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy'])
  def fit(self,train_set, valid_set):
    self.model.fit(train_set,
                    validation_data = valid_set,
                    epochs=5)
  def evaluate(self,test_set):
    return self.model.evaluate(test_set, verbose=2)
  def get_weights(self):
    i = 0 
    while i < 15:
      print('dense'+str(i))
      print(self.model.get_layer('dense'+str(i)).weights)
      i += 1
  def summary(self):
    return self.model.summary()
  def freeze(self,iteration):
    if iteration == 1:
      i = 0
      while i < 15:
        l = self.model.get_layer('dense'+str(i))
        if i not in [3, 7, 11]:
          l.trainable = False
          print('dense'+str(i)+" freezed")
        else:
          l.trainable = True
        i += 1
    elif iteration == 2:
      i = 0
      while i < 15:
        l = self.model.get_layer('dense'+str(i))
        if i not in [1, 5, 9, 13]:
          l.trainable = False
        else:
          l.trainable = True
        i += 1
    else:
      i = 0
      while i < 15:
        l = self.model.get_layer('dense'+str(i))
        if i not in [0, 2, 4, 6, 8, 10, 12, 14]:
          l.trainable = False
        else:
          l.trainable = True
        i += 1
    self.compile()
# construct model and initial weight
model = model_16()


In [ ]:
import os
model.freeze(1)
model.summary()
# print(model.get_weights())
history = model.fit(train_set_conv, valid_set_conv)
test_scores = model.evaluate(test_set_conv)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
model.summary()


dense0 freezed
dense1 freezed
dense2 freezed
dense4 freezed
dense5 freezed
dense6 freezed
dense8 freezed
dense9 freezed
dense10 freezed
dense12 freezed
dense13 freezed
dense14 freezed
Model: "model_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      [(None, 32, 32, 3)]       0         
_________________________________________________________________
flatten_34 (Flatten)         (None, 3072)              0         
_________________________________________________________________
dense0 (Dense)               (None, 500)               1536500   
_________________________________________________________________
dense1 (Dense)               (None, 500)               250500    
_________________________________________________________________
dense2 (Dense)               (None, 500)               250500    
_________________________________________________________________
dense3

In [ ]:
# model.get_weights()

dense0
[<tf.Variable 'dense0/kernel:0' shape=(3072, 100) dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Variable 'dense0/bias:0' shape=(100,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>]
dense1
[<tf.Variable 'dense1/kernel:0' shape=(100, 100) dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0

In [ ]:
model.freeze(2)
model.evaluate(test_set_conv)
model.fit(train_set_conv, valid_set_conv)

157/157 - 1s - loss: 2.0016 - accuracy: 0.2536
Epoch 1/5
704/704 [==============================] - 25s 34ms/step - loss: 2.1051 - accuracy: 0.2161 - val_loss: 2.0673 - val_accuracy: 0.2348
Epoch 2/5
704/704 [==============================] - 24s 34ms/step - loss: 2.0777 - accuracy: 0.2260 - val_loss: 2.0551 - val_accuracy: 0.2358
Epoch 3/5
704/704 [==============================] - 24s 34ms/step - loss: 2.0917 - accuracy: 0.2176 - val_loss: 2.0862 - val_accuracy: 0.2024
Epoch 4/5
704/704 [==============================] - 23s 33ms/step - loss: 2.0840 - accuracy: 0.2202 - val_loss: 2.0645 - val_accuracy: 0.2270
Epoch 5/5
704/704 [==============================] - 23s 33ms/step - loss: 2.0857 - accuracy: 0.2192 - val_loss: 2.0774 - val_accuracy: 0.2256


In [ ]:
model.freeze(3)
model.evaluate(test_set_conv)
model.fit(train_set_conv, valid_set_conv)

157/157 - 1s - loss: 2.0634 - accuracy: 0.2354
Epoch 1/5
704/704 [==============================] - 24s 33ms/step - loss: 2.3140 - accuracy: 0.1240 - val_loss: 2.3242 - val_accuracy: 0.1194
Epoch 2/5
704/704 [==============================] - 24s 34ms/step - loss: 2.3182 - accuracy: 0.1184 - val_loss: 2.3115 - val_accuracy: 0.0928
Epoch 3/5
704/704 [==============================] - 24s 34ms/step - loss: 2.3165 - accuracy: 0.1215 - val_loss: 2.3244 - val_accuracy: 0.0910
Epoch 4/5
704/704 [==============================] - 24s 34ms/step - loss: 2.3084 - accuracy: 0.1260 - val_loss: 2.2866 - val_accuracy: 0.1106
Epoch 5/5
704/704 [==============================] - 24s 34ms/step - loss: 2.3224 - accuracy: 0.1192 - val_loss: 2.2995 - val_accuracy: 0.1336


In [ ]:
# model.get_weights()

dense0
[<tf.Variable 'dense0/kernel:0' shape=(3072, 100) dtype=float32, numpy=
array([[ 1.0195618e+00, -8.8939129e-04,  1.8685929e-02, ...,
        -1.1518847e-02,  6.0755932e-03,  2.3105551e-02],
       [ 1.8395171e-02,  9.8774058e-01,  1.6620990e-03, ...,
        -8.2900384e-03,  8.9534353e-03,  2.1310752e-02],
       [ 1.6934497e-02, -1.6562000e-02,  1.0016218e+00, ...,
        -5.3506116e-03,  2.2307999e-02, -1.4602859e-02],
       ...,
       [ 4.3068267e-02,  5.3799093e-02, -3.3962194e-02, ...,
         2.3545701e-02,  2.5354395e-02, -1.4537688e-03],
       [ 2.3313347e-02,  2.7111355e-02, -2.2271283e-02, ...,
         1.5615992e-02,  4.4617239e-02,  2.0832306e-02],
       [ 4.6503525e-02,  6.0966928e-03,  2.4187867e-02, ...,
        -1.0063199e-02,  4.5627713e-02,  2.6140489e-02]], dtype=float32)>, <tf.Variable 'dense0/bias:0' shape=(100,) dtype=float32, numpy=
array([-0.00023478,  0.02074782, -0.03135146, -0.0129349 ,  0.01072712,
       -0.0127541 ,  0.01227804,  0.0009052 , -

In [ ]:
model.save("freezing1.h5")

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls

freezing1.h5  sample_data


In [ ]:

# !cp -rf lenet300100	gdrive/MyDrive/

!cp checkpoint lenet300100freezing.data-00000-of-00001 lenet300100freezing.index gdrive/MyDrive/freezing/

In [ ]:
def add_new_layer():
  inputs = keras.Input(shape=(784,), name='digits')
  dense1 = layers.Dense(784, activation='tanh', name='dense_1')
  x = dense1(inputs)
  dense2 = layers.Dense(784,kernel_initializer=tf.keras.initializers.Identity(), name='dense_2')
  x = dense2(x)
  x = tanh_layer("tanh_1")(x)
  x = layers.Dense(10, name='dense_logits_1')(x)
  outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(loss='sparse_categorical_crossentropy',
  optimizer=keras.optimizers.Adam(0.001),
  metrics=['accuracy'])
  return model

In [ ]:
model.summary()
new_model = add_new_layer()
new_model.summary()
# new_model.load_weights("freezing1.h5",by_name=True,skip_mismatch=True)
weight = new_model.get_weights()
print(weight)

Model: "model_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
tanh_1 (Activation)          (None, 784)               0         
_________________________________________________________________
dense_logits_1 (Dense)       (None, 10)                7850      
_________________________________________________________________
predictions (Activation)     (None, 10)                0         
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________
Model: "model_43"
_________________________________________________________________
Layer (type)                 Outpu

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 2.5688 - accuracy: 0.1507


[2.5687754154205322, 0.15070000290870667]

In [ ]:
history = new_model.fit(x_train, y_train,
                    batch_size=128,validation_data = (x_test,y_test),
                    epochs=10)

Epoch 1/10
430/430 [==============================] - 2s 3ms/step - loss: 0.3751 - accuracy: 0.8835 - val_loss: 0.2556 - val_accuracy: 0.9211
Epoch 2/10
430/430 [==============================] - 1s 3ms/step - loss: 0.2731 - accuracy: 0.9149 - val_loss: 0.2218 - val_accuracy: 0.9335
Epoch 3/10
430/430 [==============================] - 1s 3ms/step - loss: 0.2396 - accuracy: 0.9251 - val_loss: 0.2191 - val_accuracy: 0.9304
Epoch 4/10
430/430 [==============================] - 1s 3ms/step - loss: 0.2176 - accuracy: 0.9315 - val_loss: 0.2173 - val_accuracy: 0.9332
Epoch 5/10
430/430 [==============================] - 1s 3ms/step - loss: 0.2084 - accuracy: 0.9341 - val_loss: 0.1919 - val_accuracy: 0.9411
Epoch 6/10
430/430 [==============================] - 1s 3ms/step - loss: 0.1933 - accuracy: 0.9381 - val_loss: 0.1966 - val_accuracy: 0.9374
Epoch 7/10
430/430 [==============================] - 1s 3ms/step - loss: 0.1816 - accuracy: 0.9422 - val_loss: 0.1743 - val_accuracy: 0.9443
Epoch 

In [ ]:
new_model.evaluate(x_test, y_test, verbose=2)
print(new_model.get_weights())



313/313 - 0s - loss: 0.1682 - accuracy: 0.9471
[array([[-2.4766494e-02, -5.3947866e-02, -4.0051743e-02, ...,
         2.6188690e-02, -5.9056539e-02,  5.5198718e-02],
       [-2.6035808e-02,  3.5885908e-03, -2.2895887e-02, ...,
         1.2520295e-02,  3.9989021e-02, -1.8880997e-02],
       [ 3.7804645e-02, -3.8763508e-02, -1.0031633e-02, ...,
         5.5808011e-02, -9.8831952e-06,  4.3016646e-02],
       ...,
       [ 4.9203407e-02,  3.5709042e-02, -4.7736667e-02, ...,
        -6.0528006e-02, -5.0976183e-02, -3.5786677e-02],
       [ 1.7350476e-02, -2.4383951e-02, -3.2664016e-02, ...,
        -4.2408668e-03,  7.5398870e-03, -5.8684923e-02],
       [ 4.7757786e-02, -5.2475482e-02,  3.6039803e-02, ...,
         2.7795736e-02,  3.3402387e-02,  3.0662652e-02]], dtype=float32), array([ 3.07931118e-02, -3.52791324e-02, -3.40721048e-02,  3.22302207e-02,
        5.16012907e-02,  6.94679655e-03,  9.33919661e-03, -4.74554338e-02,
        3.00577879e-02,  5.39234616e-02, -3.45921926e-02,  5.1132

In [ ]:
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_logits_1 (Dense)       (None, 10)                7850      
_________________________________________________________________
predictions (Activation)     (None, 10)                0         
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________
